In [1]:
import numpy as np
import python_speech_features as psf
from scipy.io.wavfile import read
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import pickle

from typing import Tuple

In [2]:
male_sample = "A30003X4.wav"
female_sample = "A30002C4.wav"

male_txt = "./data/male.txt"
female_txt = "./data/female.txt"
gender_map = {"male": 0, "female": 1}

In [3]:
def get_gender_filenames(path: str) -> list:
    with open(path, "r") as f:
        filenames = f.read().splitlines()
    return filenames

In [4]:
male: list = get_gender_filenames(male_txt)
female: list = get_gender_filenames(female_txt)
genders = {"male": male, "female": female}

# Creating Dataset from files

In [5]:
def get_mfcc(filename: str) -> np.ndarray:
    samplerate, signal = read(f"data/wav_data/{filename}")
    MFCCs = psf.mfcc(signal, samplerate, 0.025, 0.01, 26, appendEnergy = False)
    mean_mfcc = np.mean(MFCCs, axis=0)
    return mean_mfcc

mfcc_sample: np.ndarray = get_mfcc(male_sample)
print(mfcc_sample, f"\n\nShape: {mfcc_sample.shape}")

[ 4.11934991e+01 -2.38203007e+01 -1.25075425e+01 -9.78714158e+00
 -7.55524300e+00 -5.24234448e+00 -6.67623883e+00 -3.74060104e-01
 -3.04456862e+00  4.42509015e+00  4.87760199e+00  5.89183918e+00
  1.17648268e-01  6.96143950e+00  7.60777816e-01  3.37629627e+00
  2.37996256e-01  2.26000480e+00 -7.83392391e-02  7.86777946e-01
 -2.09783114e-01  3.24826978e-02  1.74351193e-02 -2.85513283e-02
 -7.24971332e-02 -1.78644926e-01] 

Shape: (26,)


In [6]:
def create_dataset(source_dict: dict) -> Tuple[np.ndarray, np.ndarray]:
    dataset = []
    Y = np.array([])
    for gender, filenames in source_dict.items():
        for file in filenames:
            mfcc = get_mfcc(file)
            dataset.append(mfcc)
        gender_vector = gender_map[gender] * np.ones(len(filenames))
        Y = np.append(Y, gender_vector)
    return np.array(dataset), Y

In [7]:
X, Y = create_dataset(genders)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
def test_classifier(classifier, test_x, test_y):
    
    accuracy = classifier.score(test_x,test_y)
    print("Test accuracy: ", accuracy)

    predictions = classifier.predict(test_x)

    male_as_female = np.sum(np.logical_and(test_y==1, predictions==0))
    female_as_male = np.sum(np.logical_and(test_y==0, predictions==1))
    print("\n{:d} males classified as females out of {:.0f}, {:.3f} %".format(male_as_female, np.sum(test_y==1), 100 * male_as_female / np.sum(test_y==1)))
    print("{:d} females classified as males out of {:.0f}, {:.3f} %".format(female_as_male, np.sum(test_y==0), 100 * female_as_male / np.sum(test_y==0)))

    print(classification_report(test_y, predictions))

# Creating and training a **GaussianNB**

In [9]:
model = GaussianNB()
model.fit(X_train, y_train)

train_accuracy = model.score(X_train, y_train)
print("Training Accuracy : ", round(train_accuracy, 3))
test_classifier(model, X_test, y_test)

Training Accuracy :  0.982
Test accuracy:  1.0

0 males classified as females out of 19, 0.000 %
0 females classified as males out of 22, 0.000 %
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        22
         1.0       1.00      1.00      1.00        19

    accuracy                           1.00        41
   macro avg       1.00      1.00      1.00        41
weighted avg       1.00      1.00      1.00        41



# Save model

In [10]:
filename = 'GaussianNB.sav'
pickle.dump(model, open(filename, 'wb'))